In [1]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import re
import os
import glob

In [2]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
TAXI_ZONES = "taxi_zones/taxi_zones.shp"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber/uber_rides_sample.csv"
WEATHER_PATH = "weather"


NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
def find_taxi_parquet_urls(taxi_url):
    response = requests.get(taxi_url)
    html = response.content
    results_page = bs4.BeautifulSoup(html, 'html.parser')
    links = results_page.find_all('a',title=r'Yellow Taxi Trip Records')
    pattern = r"yellow_tripdata_(2009|201[0-4]|2015-0[1-6])"
    urls = [link.get('href') for link in links if re.search(pattern,link.get('href'))]
    return urls

In [4]:
def get_month_taxi_data(taxi_data_url):
    pattern = r'yellow_tripdata_[0-9]{4}-[0-9]{2}'
    name = re.search(pattern,taxi_data_url).group()
    path = os.path.join(os.getcwd(),'taxi',name+'.csv')
    if os.path.exists(path):
        print("read")
        df = pd.read_csv(path)
    else:
        print("download",str(name))
        df = pd.read_parquet(taxi_data_url, engine='pyarrow')
        df.to_csv(path)
    return df

In [5]:
def sample_taxi_data(row_taxi_data,taxi_data_url):
    pattern = r'yellow_tripdata_[0-9]{4}-[0-9]{2}'
    name = re.search(pattern,taxi_data_url).group()
    path = os.path.join(os.getcwd(),'sample_taxi',name+'.csv')
    if os.path.exists(path):
        print("exist")
        taxi_data = pd.read_csv(path)
    else:
        print("sample",str(name))
        taxi_data = row_taxi_data.sample(3000,random_state=1)
        taxi_data.to_csv(path)
    return taxi_data

In [6]:
def get_and_clean_taxi_data():
    all_csv_urls = find_taxi_parquet_urls(TAXI_URL)
    taxi_zone_df = gpd.read_file(TAXI_ZONES)
    taxi_zone_df = taxi_zone_df.to_crs(4326)
    taxi_zone_df['lon'] = taxi_zone_df.centroid.x  
    taxi_zone_df['lat'] = taxi_zone_df.centroid.y
    all_taxi_dataframes = []
    for csv_url in all_csv_urls:
        month_taxi_dataframe = get_month_taxi_data(csv_url)
        month_taxi_dataframe = sample_taxi_data(month_taxi_dataframe,csv_url)
        all_taxi_dataframes.append(month_taxi_dataframe)
    all_taxi_data = pd.concat(all_taxi_dataframes)
    return all_taxi_data

In [7]:
taxi_data = get_and_clean_taxi_data()
taxi_data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14464\389512193.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone_df['lon'] = taxi_zone_df.centroid.x
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14464\389512193.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone_df['lat'] = taxi_zone_df.centroid.y


read
sample yellow_tripdata_2015-01
read
sample yellow_tripdata_2015-02
read
sample yellow_tripdata_2015-03
read
sample yellow_tripdata_2015-04
read
sample yellow_tripdata_2015-05
read
sample yellow_tripdata_2015-06
read
sample yellow_tripdata_2014-01
read
sample yellow_tripdata_2014-02
read
sample yellow_tripdata_2014-03
read
sample yellow_tripdata_2014-04
read
sample yellow_tripdata_2014-05
read
sample yellow_tripdata_2014-06
read
sample yellow_tripdata_2014-07
read
sample yellow_tripdata_2014-08
read
sample yellow_tripdata_2014-09
read
sample yellow_tripdata_2014-10
read
sample yellow_tripdata_2014-11
read
sample yellow_tripdata_2014-12
read
sample yellow_tripdata_2013-01
read
sample yellow_tripdata_2013-02
read
sample yellow_tripdata_2013-03
read
sample yellow_tripdata_2013-04
read
sample yellow_tripdata_2013-05
read
sample yellow_tripdata_2013-06
read
sample yellow_tripdata_2013-07
read
sample yellow_tripdata_2013-08
read
sample yellow_tripdata_2013-09
read
sample yellow_tripdata_

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14464\1280330153.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


sample yellow_tripdata_2010-01
read


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14464\1280330153.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


...

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14464\1280330153.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


sample yellow_tripdata_2010-11
read


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14464\1280330153.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


sample yellow_tripdata_2010-12
read
sample yellow_tripdata_2009-01
read
sample yellow_tripdata_2009-02
read
sample yellow_tripdata_2009-03
read
sample yellow_tripdata_2009-04
read
sample yellow_tripdata_2009-05
read
sample yellow_tripdata_2009-06
read
sample yellow_tripdata_2009-07
read
sample yellow_tripdata_2009-08
read
sample yellow_tripdata_2009-09
read
sample yellow_tripdata_2009-10
read
sample yellow_tripdata_2009-11
read
sample yellow_tripdata_2009-12


,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,Tip_Amt,Tolls_Amt,Total_Amt
4855757,4855757,1.0,2015-01-12 23:07:25,2015-01-12 23:16:05,1.0,1.50,1.0,N,161.0,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3760945,3760945,1.0,2015-01-10 13:07:59,2015-01-10 13:16:10,1.0,1.20,1.0,N,230.0,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10779875,10779875,1.0,2015-01-26 19:03:56,2015-01-26 19:08:41,2.0,0.70,1.0,N,238.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2237074,2237074,2.0,2015-01-07 06:49:32,2015-01-07 06:54:15,1.0,0.80,1.0,N,186.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7110478,7110478,2.0,2015-01-17 22:40:14,2015-01-17 22:53:53,4.0,1.51,1.0,N,79.0,249.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9797679,9797679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.750732,NaN,NaN,-73.991467,40.750545,CASH,2.5,0.0,0.0,3.0
11019086,11019086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.757572,NaN,NaN,-74.009675,40.710643,CASH,14.1,0.0,0.0,14.6
13955982,13955982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.751533,NaN,NaN,-73.963287,40.765978,CASH,9.3,0.0,0.0,9.8
14345414,14345414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.779707,NaN,NaN,-73.968960,40.764173,CASH,13.3,0.0,0.0,14.8


In [8]:
taxi_data.columns

Index(['Unnamed: 0', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee', 'vendor_id', 'pickup_datetime',
       'dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'rate_code',
       'dropoff_longitude', 'dropoff_latitude', 'surcharge', 'vendor_name',
       'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Passenger_Count',
       'Trip_Distance', 'Start_Lon', 'Start_Lat', 'Rate_Code',
       'store_and_forward', 'End_Lon', 'End_Lat', 'Payment_Type', 'Fare_Amt',
       'Tip_Amt', 'Tolls_Amt', 'Total_Amt'],
      dtype='object')